In [174]:
from dotenv import load_dotenv

In [175]:
load_dotenv()

True

In [176]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [177]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [134]:
embeddings.embed_query("hello AI")

[-0.03338824212551117,
 0.03453981503844261,
 0.05947450175881386,
 0.059286098927259445,
 -0.06353531032800674,
 -0.06819586455821991,
 0.08823321014642715,
 0.0344407856464386,
 -0.03278516232967377,
 -0.01581495814025402,
 0.020981721580028534,
 -0.018340323120355606,
 -0.03983224928379059,
 -0.0804707333445549,
 -0.014469229616224766,
 0.03326485678553581,
 0.014259278774261475,
 -0.03404996171593666,
 -0.142915740609169,
 -0.023083431646227837,
 -0.021380223333835602,
 0.002633583964779973,
 -0.04729269817471504,
 -0.010752756148576736,
 -0.06866802275180817,
 0.031124936416745186,
 0.07594586908817291,
 0.0011282607447355986,
 0.011632048524916172,
 -0.036039240658283234,
 0.04483754187822342,
 0.018390731886029243,
 0.12672801315784454,
 -0.0013597395736724138,
 0.00820669624954462,
 0.06909967958927155,
 -0.08076362311840057,
 -0.05841310694813728,
 0.0537545382976532,
 0.02622750587761402,
 -0.0068285781890153885,
 -0.05635844171047211,
 0.0032929808367043734,
 -0.072501882910

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
documents=["what is benefit of green tea?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [25]:
my_query="Narendra modi is prime minister of india?"

In [26]:
document_embedding=embeddings.embed_documents(documents)

In [27]:
document_embedding

[[-0.05074392259120941,
  0.07871920615434647,
  0.02454959973692894,
  -0.016062268987298012,
  0.09389403462409973,
  0.12062039971351624,
  0.0756671279668808,
  0.020446613430976868,
  0.002522907918319106,
  0.01198961678892374,
  -0.012409032322466373,
  -0.007632073014974594,
  -0.03452015668153763,
  -0.031952641904354095,
  0.047444116324186325,
  0.037579867988824844,
  0.0024465536698698997,
  0.04336089268326759,
  -0.06804467737674713,
  -0.11454377323389053,
  -0.0936029776930809,
  -0.003596389899030328,
  0.04159275442361832,
  0.024034399539232254,
  -0.03245614469051361,
  0.03021431900560856,
  -0.011403923854231834,
  -0.017884716391563416,
  0.018772805109620094,
  -0.0011700880713760853,
  0.022109434008598328,
  0.03752337768673897,
  0.008077813312411308,
  -0.007398846559226513,
  -0.1716674268245697,
  0.037111349403858185,
  0.06671693176031113,
  -0.08183003216981888,
  -0.01989125832915306,
  -0.05425114929676056,
  -0.05056340619921684,
  -0.02310291305184

In [28]:
query_embedding=embeddings.embed_query(my_query)

In [29]:
len(query_embedding)

384

In [30]:
cosine_similarity([query_embedding],document_embedding)

array([[0.08588358, 0.34324566, 0.81413235]])

In [31]:
from sklearn.metrics.pairwise import euclidean_distances

In [33]:
euclidean_distances([query_embedding], document_embedding)

array([[1.35212164, 1.14608409, 0.60970098]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [36]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [37]:
index=faiss.IndexFlatL2(384)


In [38]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000203CDE98D50> >

In [58]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [59]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['39db05ed-a5a5-404e-83f7-7ef0117f2a94',
 '76942ca6-9e59-4458-ac10-35b1aefa9d38',
 '39f6a6f6-9ede-4bdc-8a70-51a12d66a7a9']

In [60]:
vector_store.index_to_docstore_id

{0: '39db05ed-a5a5-404e-83f7-7ef0117f2a94',
 1: '76942ca6-9e59-4458-ac10-35b1aefa9d38',
 2: '39f6a6f6-9ede-4bdc-8a70-51a12d66a7a9'}

In [64]:
results = vector_store.similarity_search("Tell me about AI",k=1)

In [66]:
results


[Document(id='76942ca6-9e59-4458-ac10-35b1aefa9d38', metadata={}, page_content='AI is powerful')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [67]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [68]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [69]:
vector_store.add_documents(documents=documents)

['2b9c3460-ef3f-4f2a-95a0-a5a9fed76a19',
 '1d8c6df2-d193-4049-afa9-d1993e695367',
 '819298a9-1435-42cd-a26a-6dbabe82c3b8',
 '9a16b513-bc32-42d3-9f03-f11e1af3a30b',
 '56202cf0-3a31-423f-99fb-edf945e2e759',
 'bcf0f95d-4503-474b-93fd-844ee24b5a42',
 'c2030945-ecae-4ce1-af4d-0de5adf7ff0a',
 'f7ca9ba9-571b-4af6-891c-652bd0b961d2',
 '3a5b6e32-0dfe-4f0d-a6b5-1c632d38523a',
 'dd128177-6d37-4da2-97e6-157634cafba5']

In [ ]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter num of documents to return
    
)

[Document(id='dd128177-6d37-4da2-97e6-157634cafba5', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='3a5b6e32-0dfe-4f0d-a6b5-1c632d38523a', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [82]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    # k=2, #hyperparameter
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='819298a9-1435-42cd-a26a-6dbabe82c3b8', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7ca9ba9-571b-4af6-891c-652bd0b961d2', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='dd128177-6d37-4da2-97e6-157634cafba5', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='2b9c3460-ef3f-4f2a-95a0-a5a9fed76a19', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [83]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [89]:
result[0].metadata

{'source': 'news'}

In [92]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [93]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [94]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='819298a9-1435-42cd-a26a-6dbabe82c3b8', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7ca9ba9-571b-4af6-891c-652bd0b961d2', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='dd128177-6d37-4da2-97e6-157634cafba5', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [ ]:
inmemory(server)
ondisk(server)
cloud(yet to discuss)

In [95]:
vector_store.save_local("today's class faiss index")

In [96]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [98]:
new_vector_store.similarity_search("langchain")

[Document(id='819298a9-1435-42cd-a26a-6dbabe82c3b8', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='f7ca9ba9-571b-4af6-891c-652bd0b961d2', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='56202cf0-3a31-423f-99fb-edf945e2e759', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='c2030945-ecae-4ce1-af4d-0de5adf7ff0a', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [135]:
from langchain_community.document_loaders import PyPDFLoader

In [136]:
FILE_PATH=r"C:\Projects-Agentic\Agentic AI\Agentic-AI\1-Langchain-Pydantic\syllabus.pdf"

In [137]:
loader=PyPDFLoader(FILE_PATH)

In [138]:
len(loader.load())

34

In [139]:
pages=loader.load()

In [140]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [141]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [142]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [143]:
split_docs = splitter.split_documents(pages)

In [144]:
len(split_docs)

81

In [145]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [146]:
vector_store.add_documents(documents=split_docs)

['eb057dd2-5b4f-41d2-9fa5-9f7b4347af22',
 '3d281b61-dc93-47b6-940e-8d3b165c70df',
 '57dfc900-6708-402e-9898-91eace0afaaa',
 'e6c6286a-ce11-4881-8f9d-ed5ed0bb7e52',
 '1958d520-8a12-4d3b-89ea-28d3374f8016',
 'dbc3e885-1266-4b9b-82cd-b6a14409f03c',
 '115654cc-6022-41e5-bb48-6e2a60592507',
 'be7296c0-aeca-457e-897f-1b3879b597df',
 '7f426ac5-9c0e-4742-8520-d2374a893b1b',
 '8def7833-d513-4b49-ac56-f7a052cd4173',
 'eae623ff-7a27-4933-835d-e1d860529414',
 '9db74a4c-1465-4925-ab7a-2ab624b6fa78',
 '49efa2a1-8d2c-4ca6-acf7-fc23c9ae5455',
 '01366a65-eb4f-413f-9d98-2f76640c3d47',
 '1b34b82d-5ab2-4fcc-891f-991064239acc',
 'ae53e2de-5de2-4b0f-9055-dc9dc1229a4b',
 '4ffc4d0c-bb18-41e9-a40d-ae4fa5f688af',
 'aca4ecc5-52be-4cea-860c-6b9365d80c82',
 'a70bf157-2a5f-4805-96c8-95dbc18cb708',
 'ca88b2fd-785e-4a3b-b465-5b3d64043ae3',
 '7134c793-8c83-4e3c-8ba0-f6e4e5c2fd99',
 'f30003ac-cf6c-4f06-bbc5-a5891505f26a',
 'e235ad3c-c5ec-4c7b-9f37-dc7dde539746',
 '8f11b11c-5b8f-475a-9e60-b7621ec4d3d9',
 '613b7dae-72ad-

In [147]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [148]:
retriever.invoke("who is Krish?")

[Document(id='7f426ac5-9c0e-4742-8520-d2374a893b1b', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-01-30T20:27:03+00:00', 'title': 'Ultimate Data Science & GenAI Bootcamp', 'moddate': '2025-01-30T20:26:59+00:00', 'keywords': 'DAGdmhcqnYw,BAEmsmap8Lg,0', 'author': 'monal singh', 'containsaigeneratedcontent': 'Yes', 'source': 'C:\\Projects-Agentic\\Agentic AI\\Agentic-AI\\1-Langchain-Pydantic\\syllabus.pdf', 'total_pages': 34, 'page': 3, 'page_label': '4'}, page_content='Course Instructors\nSunny SavitaGenAI Engineer\nLinkedin\nKrish NaikChief AI Engineer\nLinkedin\nUltimate Data Science & GenAI Bootcamp       Page  4\nSourangshu PalSenior Data Scientist\nLinkedin\nMonal KumarData Scientist\nLinkedin\nMayank AggrawalSenior ML Engineer\nLinkedin\nDarius B.Head of Product\nLinkedin'),
 Document(id='14571051-289d-4491-b97e-47134b999794', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-01-30T20:27:03+00:00', 'title': 'Ultimate Data Science 

In [165]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [166]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="o1-mini")

In [167]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [168]:
import pprint


In [169]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [170]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [126]:
context(retriever),prompt(hub),model(google),parser(langchain)

NameError: name 'context' is not defined

In [171]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [178]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [179]:
rag_chain.invoke("who is Krish?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}